<h1> OCR - Scrape PNG Screenshot Site Data </h1>

In [ ]:
# data import and file manipulation
import os
import requests
from requests.structures import CaseInsensitiveDict
import json
import csv
import xlrd

# data scraping
from PIL import Image
import pytesseract
import cv2
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
import time


#data conditioning
import pandas as pd
import numpy as np
import re
import datetime as dt
from bs4 import BeautifulSoup

#data visualization
[REDACTED]/.pyplot as plt
import seaborn as sns

In [ ]:
 # add current timestamp to filename for reference
current_time = (dt.datetime.utcnow().strftime('%Y_%m_%d_%H%M%S'))

# git repo folder
git_folder = 'd:/git/example_infrastructure_data_dev'

# dictionary Directory
dictionary_dir = 'd:/git/example_infrastructure_data_dev/dictionaries'

# source dir for nable exported data
source_folder = 'd:/project_docs/abc_nable_migration/abc_nable_exports/patch_management'

# export folder will contain all csv exported DataFrames for Ticket Creation
export_folder = 'd:/exports'

In [ ]:
# import configparser for env secrets
from configparser import ConfigParser

config = ConfigParser()
config.read(f'{git_folder}/config/env.ini')
import requests
from requests.structures import CaseInsensitiveDict

## Create Chrome Driver

In [ ]:
chrome_driver = 'd:/web_drivers/chrome/chromedriver_win32/chromedriver.exe'
options = webdriver.ChromeOptions()
options.add_argument('headless')
c_driver = webdriver.Chrome(executable_path=chrome_driver)

## Request Header Info (if needed to impersonate api browser calls)

In [ ]:
# construct header
headers = CaseInsensitiveDict()
headers['User-Agent'] = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'

# construct req body
data = CaseInsensitiveDict()
data = ''

In [ ]:
baseUrl = 'https://ncod405.n-able.com/'
auth_url = f'https://ncod405.n-able.com/so/abc'

links = []

# Login to N-Able

In [ ]:
nable_config = config['nable']

In [ ]:
c_driver.get(auth_url)

In [ ]:
title = c_driver.find_element_by_xpath('//body')
title.text

In [ ]:
login_txtbox = c_driver.find_element_by_xpath("//input[@name='username']")

In [ ]:
login_txtbox.send_keys((nable_config['username']),Keys.ENTER)

In [ ]:
pass_txtbox = c_driver.find_element_by_xpath("//input[@id='passwordFieldId']")

# Patch Managment Summary Page

In [ ]:
# Click on patch management button
patch_mgmt_button = c_driver.find_element_by_xpath("/html/body/div[1]/div[3]/div[7]/div[2]/div/div/ul/li[7]/a")
patch_mgmt_button.click()

In [ ]:
# Get 'Patch Enabled Rules' section
patch_enabled_rules = c_driver.find_element_by_xpath("/html/body/div[1]/div[2]/div[1]/div/div[4]/div/div[2]/div/div")

In [ ]:
policies_list = []
headings = ['ruleName','so','clientName','site','associatedProfile','devices']
i = 0
policy_dict = {}
for span in patch_enabled_rules.find_elements_by_xpath(".//span"):
    item_dict = {}
    if span.text != '':
        print(f'assigning {headings[i]} to {span.text}')

        item_dict[headings[i]] = span.text
        policy_dict.update(item_dict)
        i = i+1
        if i == len(headings):
            policies_list.append(policy_dict)
            policy_dict = {}
            i = 0

In [ ]:
policies_list

In [ ]:
df_patch_enabled_rules = pd.DataFrame(policies_list)

In [ ]:
df_patch_enabled_rules

In [ ]:
df_patch_enabled_rules.to_csv('[REDACTED]/.csv')

## Pull Patch Policy Application Info

In [ ]:
# !Patch: Servers (Windows) Patch Group 1 (Packerland)
devices_table = c_driver.find_element_by_xpath("/html/body/div[5]/div[2]/div/div[1]")

policies_list = []
headings = ['Customer',
            'Site',
            'Device Name',
            'Network Address',
            'Device Class',
            'Operating System'
            ]
i = 0
policy_dict = {}
for td in devices_table.find_elements_by_xpath(".//td"):
    item_dict = {}
    if td.text != '':
        print(f'assigning {headings[i]} to {td.text}')

        item_dict[headings[i]] = td.text
        policy_dict.update(item_dict)
        i = i+1
        if i == len(headings):
            policies_list.append(policy_dict)
            policy_dict = {}
            i = 0

In [ ]:
df_current = pd.DataFrame(policies_list)

In [ ]:
df_current

In [ ]:
df_current.to_csv('[REDACTED]/.csv',index=False)

In [ ]:
df_patch_enabled_rules

In [ ]:
c_driver.execute_script ("window.scrollBy(0,-250)")

In [ ]:

for root, dirs, files in os.walk('D:\exports\patch_policy_scope_exports'):
    merge_exists = False
    for file in files:
        print(merge_exists)
        root_loc = re.sub(r'\\','/',root)
        file_loc = (f"{root_loc}/{file}")
        policy_name = re.findall(r'([^\.]+)[REDACTED]/.csv',file)
        print(f'Attempting to read file loc: {file_loc}')
        print(policy_name)
        df = pd.read_csv(file_loc)
        df['policy'] = policy_name[0]
        while(merge_exists == False):
            df_merged = df
            merge_exists = True
        df_merged = pd.concat([df, df_merged],ignore_index=True)







In [ ]:
df_merged = df_merged[['Customer','Site','Device Name','Network Address','Device Class','Operating System','policy']]

In [ ]:
df_merged

In [ ]:
df = pd.read_csv("[REDACTED]/.csv")

In [ ]:
df_merged = pd.merge(df_current, df_merged, on=['Device Name'], how='outer')

In [ ]:
df_merged.drop_duplicates(subset=['Device Name'], keep='first', inplace=True, ignore_index=True)

In [ ]:
df_merged